# SerpAPI Google Scholar Search

**Author:** Jack Galbraith-Edge

**Date:** 8th January 2025

## Background
Google Scholar is popular tool for searching grey literature. Unlike databases like PubMed and Embase which are curated, maintained by humans and contain only published literature. Google Scholar, on the other hand, searches the internet for what it believes are academic materials and then displays them in a list. As a result, Google Scholar is excellent for seaching grey literature, as results will include unpublished data such as thesises and other useful material, reducing publication bias and susceptability bias (Haddaway et al., 2015)

Unfortunately, because Google Scholar references the internet in this way and is not itself a database, this can make querying cumbersome and the number of results returned by queries and be overwhelming. 

Following the failure of my experiments with Scholarly, I opted to use SerpAPI to query Google Scholar and clean and manipulate the results to add to my systematic review database/literature search. SerpAPI has a free-tier up to 100 Google Scholar requests per month, so I utilise my free trial and give it a go.

This document details my search queries and how I've tailored them to gather an appropriate number of results.

## Important
The result limit in my query function is set to 300. This is because some research exists to researches should focus on the first 200-300 results, with useful resources being found after the 80 result mark (Haddaway et al., 2015).

## Setup

In [16]:
# import libraries
import os   # for navigating file system
import json # for working with json files
from dotenv import load_dotenv  # to load enviromental variables and aid privacy of API keys and similar
from serpapi import GoogleSearch    # for querying google scholar
import pandas as pd # for working with dataframes

In [17]:
# load environmental variables
load_dotenv()

# get SerpAPI key from .env file.
try: 
    SERP_API_KEY = os.getenv('SERP_API_KEY')
    print("SerpAPI key loaded.")
except KeyError:
    print("SerpAPI Key not found")

# get file output path from .env
try:
    OUTPUT_PATH = os.getenv('GS_SEARCHES_OUTPUT_PATH')
    print("OUTPUT_PATH loaded.")
except KeyError:
    print("Error: OUTPUT_PATH not found in .env file")

# get file output path from .env
try:
    INPUT_PATH = os.getenv('GS_SEARCHES_INPUT_PATH')
    print("INPUT_PATH loaded.")
except KeyError:
    print("Error: INPUT_PATH not found in .env file")

SerpAPI key loaded.
OUTPUT_PATH loaded.
INPUT_PATH loaded.


## Functions

In [18]:
# define a function to query google scholar
def query_google_scholar(google_scholar_query, api_key=SERP_API_KEY, max_results=300):

    """
    Query google scholar using SerpAPI
    
    Parameters: 
        - google_scholar_query: a string search query
        - api_key = SerpAPI key, available at: https://serpapi.com/
        - max_results = 300. Set at this level as single author and some evidence says that this is suitable limit.
    
    - Sets language to english
    
    """

    all_results = []  # store all results here
    start = 0         # start with the first page

    while len(all_results) < max_results:
        # define parameters for searching google scholar
        params = {
          "api_key": api_key,  # SerpAPI key
          "engine": "google_scholar", # google_scholar engine
          "q": google_scholar_query, # define query
          "start": start,
          "num": 20,
          "hl": "en" # set language to english
        }

        # search google scholar
        search = GoogleSearch(params)
        results = search.get_dict() # get results as dictionary

        # get organic results
        organic_results = results.get("organic_results", []) # retrieve organic list of results from results

        # add results to the list
        all_results.extend(organic_results)

        # check if there are no more results
        if len(organic_results) < 20:
            print("No more results to fetch.")
            break  # exit the loop

        # increment parameter for next page
        start += 20
        print(f"Fetched {len(all_results)} results so far...")

        if len(all_results) == max_results:
            print(f"Maximum number of {max_results} results reached.")

    return all_results[:max_results]

In [19]:
def scholar_to_df(organic_results, all_results_dict):

    """
    A function to transform dictionary output of SerpAPI Google Scholar Query into a
    pandas dataframe.

    Requirements:
        - Pandas

    Usage: 
        - Takes input from query_google_scholar function.
        - Outputs a pandas DataFrame
    """
# convert organic results to dictionary
    organic_results_dict = {}

    for item in organic_results:
        organic_results_dict[len(organic_results_dict)] = item

    organic_results_df = pd.DataFrame(columns=[
    'Publication Year', 'First Author', 'Summary', 'Authors', 'Publication Title',
    'Title', 'Abstract', 'URL', 'Database', 'Exclude', 'Reason ID',
    'Reason'])

    # create empty list to store rows
    rows = []

    # iterate through organic_results_dict
    for item in organic_results_dict.values():
        # extract information
        title = item.get('title', None)
        abstract = item.get('snippet', None)

        summary = item['publication_info'].get('summary', "Summary not found.")
        if summary:
            authors = summary.split(' - ')[0].strip() if ' - ' in summary else "No Author Listed"
            publication_info = summary.split(' - ')[1].strip() if ' - ' in summary else "No Publication Information"
            publication_title = publication_info.split(',')[0].strip() if ',' in publication_info else None
            year = publication_info.split(',')[-1].strip() if ',' in publication_info else None
        else:
            authors = publication_title = year = None

        # add database specific fields
        database = "Google Scholar"
        exclude = None
        reason_id = None
        reason = None

        # append row as dictionary
        rows.append({
            'Publication Year': year,
            'First Author': f"{authors.split(' ')[-1]}, {authors.split(' ')[0][0]}." if authors else None,
            'Authors': authors,
            'Summary': summary,
            'Publication Title': publication_title,
            'Title': title,
            'Abstract': abstract, 
            'URL': item.get('link', None),
            'Database': database,
            'Exclude': exclude,
            'Reason ID': reason_id,
            'Reason': reason
        })

    organic_results_df = pd.DataFrame(rows).sort_values(by="Publication Year", ascending=True).reset_index(drop=True)

    return organic_results_df

In [20]:
# append to all results dictionary
def append_results(query, df, results_dict):

    """
    A function that appends query and the resultant dataframe to a dictionary.
    Allows for tracking of queries and for number of results from these queries. 
    """

    if results_dict is None:
        results_dict = {}

    results_dict[len(results_dict)] = {
        "query": query,
        "results": df}
    return results_dict

## Queries

In [21]:
# create dictionary to store search query and results in for ease of reference later.
all_results_dict = {}

### Query 1

In [22]:
# Define google scholar query
google_scholar_query =  """
                        ("foreign obj*" OR "foreign bod*")
                        """

In [23]:
organic_results = query_google_scholar(google_scholar_query)


Fetched 20 results so far...
Fetched 40 results so far...
Fetched 60 results so far...
Fetched 80 results so far...
Fetched 100 results so far...
Fetched 120 results so far...
Fetched 140 results so far...
Fetched 160 results so far...
Fetched 180 results so far...
Fetched 200 results so far...
Fetched 220 results so far...
Fetched 240 results so far...
Fetched 260 results so far...
Fetched 280 results so far...
Fetched 300 results so far...
Maximum number of 300 results reached.


In [24]:
google_scholar_results_df = scholar_to_df(organic_results, all_results_dict)

# append results to final
all_results_dict = append_results(google_scholar_query, google_scholar_results_df, all_results_dict)

### Query 2

In [25]:
# Define google scholar query
google_scholar_query =  """
                        ("foreign obj*" OR "foreign bod*")
                        AND
                        ("intent*" OR "deliberate*" OR "purpose*" OR "self-injur*" OR "selfharm*" OR "self-harm*")
                        """

In [26]:
# query google scholar
organic_results = query_google_scholar(google_scholar_query)

Fetched 20 results so far...
Fetched 40 results so far...
Fetched 60 results so far...
Fetched 80 results so far...
Fetched 100 results so far...
Fetched 120 results so far...
Fetched 140 results so far...
Fetched 160 results so far...
Fetched 180 results so far...
Fetched 200 results so far...
Fetched 220 results so far...
Fetched 240 results so far...
Fetched 260 results so far...
Fetched 280 results so far...
Fetched 300 results so far...
Maximum number of 300 results reached.


In [27]:
google_scholar_results_df = scholar_to_df(organic_results, all_results_dict)

# append results to final
all_results_dict = append_results(google_scholar_query, google_scholar_results_df, all_results_dict)

### Query 3

In [28]:
# Define google scholar query
google_scholar_query =  """
                        ("foreign obj*" OR "foreign bod*")
                        AND
                        ("intent*" OR "deliberate*" OR "purpose*" OR "self-injur*" OR "selfharm*" OR "self-harm*")
                        AND
                        ("ingest*" OR "swallow*")
                        """

In [29]:
# query google scholar
organic_results = query_google_scholar(google_scholar_query)

Fetched 20 results so far...
Fetched 40 results so far...
Fetched 60 results so far...
Fetched 80 results so far...
Fetched 100 results so far...
Fetched 120 results so far...
No more results to fetch.


In [30]:
google_scholar_results_df = scholar_to_df(organic_results, all_results_dict)

# append results to final
all_results_dict = append_results(google_scholar_query, google_scholar_results_df, all_results_dict)

### Query 4

In [31]:
# Define google scholar query
google_scholar_query =  """
                        ("foreign obj*" OR "foreign bod*")
                        AND
                        ("intent*" OR "deliberate*" OR "purpose*" OR "self-injur*" OR "selfharm*" OR "self-harm*")
                        AND
                        ("ingest*" OR "swallow*"))
                        AND
                        ("surg*" OR "endoscop*" OR "EGD" OR "OGD" OR "Esophagogastroduodenoscopy" OR "Oesophagogastroduodenoscopy" OR "manag*")"
                        """

In [32]:
# query google scholar
organic_results = query_google_scholar(google_scholar_query)

Fetched 20 results so far...
Fetched 40 results so far...
Fetched 60 results so far...
No more results to fetch.


In [33]:
google_scholar_results_df = scholar_to_df(organic_results, all_results_dict)

# append results to final
all_results_dict = append_results(google_scholar_query, google_scholar_results_df, all_results_dict)

### Query 5

((deliberate OR purpose OR "self-injur*" OR "selfharm*" OR "self-harm*”)
AND (ingest* OR swallow)) AND.... remaining keyword search terms in one line

SO

("intent*" OR "deliberate" OR "purpose*" OR "self-injur*" OR "selfharm*" OR "self-harm*") AND (ingest* OR swallow*) AND ("foreign obj*" OR "foreign bod*" OR surg* OR endoscop* OR EGD OR OGD OR Esophagogastroduodenoscopy OR Oesophagogastroduodenoscopy OR manag*)

OR



### Query Summary

In [34]:
# show number of results for each query and store in dataframe

# create results dataframe
results_df = pd.DataFrame(columns=["Query", "Num Results"])

# initialise list to store rows in 
rows = []

# iterate through all_results_dict that contains queries and results dataframes from query
for item in all_results_dict.values():
    
    query = item.get('query') # get query
    num_results = len(item.get('results')) # get number of results - max is 300.

    # append row as dictionary
    rows.append({
        "Query": query,
        "Num Results": num_results
        })

# create dataframe from rows
results_df = pd.DataFrame(rows)

results_df

,Query,Num Results
0,"\n (""foreign obj*"" OR ""...",300
1,"\n (""foreign obj*"" OR ""...",300
2,"\n (""foreign obj*"" OR ""...",134
3,"\n (""foreign obj*"" OR ""...",60


In [35]:
def all_results_to_csv(results, output_path):
    """
    Function that exports dataframes from queries to CSV.

    Parameters:
        - Results dictionary
    """
    # check results are in dictionary format
    if not isinstance(results, dict):
        raise TypeError(f"Expected a dictionary, but got {type(result).__name__} instead.")

    # export results to csv to inspect
    for n, value in enumerate(results.values()):
        df = value['results']
        output_file = f"{output_path}/google_scholar_results_{n}.csv"
        df.to_csv(output_file, index=False)

all_results_to_csv(all_results_dict, OUTPUT_PATH)

In [36]:
# define query to export results from all_results_dict as csv files.
def results_to_csv(results, output_path):
    """
    Export query and query counts to a CSV file.

    Parameters:
        - results: A pandas DataFrame.
        - output_path: A string representing the output file path (including the file name).
    """
    # Check that results is a pandas dataframe
    if not isinstance(results, pd.DataFrame):
        raise TypeError(f"Expected a DataFrame, but got {type(results).__name__} instead.")
    
    # strip out \n from all cells in the dataframe
    results = results.apply(lambda col: col.map(lambda x: x.replace('\n', '') if isinstance(x, str) else x))
    
    output_file = f"{output_path}/google_query_results_counts.csv"

    # Save the DataFrame to CSV
    results.to_csv(output_file, index=False)

In [37]:
# export queries and associated results count to review with supervisor.
results_to_csv(results_df, OUTPUT_PATH)


## Clean

In [40]:
# import google_df from previously exported csv
google_df = pd.read_csv("/".join([INPUT_PATH, "google_scholar_results_2.csv"]))

google_df['First Author'] = google_df['First Author'].str.title() # clean first author names
google_df['Publication Title'] = google_df['Publication Title'].str.title() 
google_df['Title'] = google_df['Title'].str.title()
google_df['URL'] = google_df['URL'].str.lower()

google_df

,Publication Year,First Author,Authors,Summary,Publication Title,Title,Abstract,URL,Database,Exclude,Reason ID,Reason
0,1886.0,"Richardson, M.",MH RICHARDSON,MH RICHARDSON - The Boston Medical and Surgica...,The Boston Medical And Surgical Journal,A Case Of Gastrotomy. Digital Exploration Of Œ...,Many cases have been recorded where foreign bo...,https://www.nejm.org/doi/pdf/10.1056/nejm18861...,Google Scholar,NaN,NaN,NaN
1,1895.0,"Morrison, W.",WA MORRISON,WA MORRISON - The Boston Medical and Surgical ...,The Boston Medical And Surgical Journal,The Value Of The Stomach-Tube In Feeding After...,By this method I fed the child with the tube f...,https://www.nejm.org/doi/pdf/10.1056/nejm18950...,Google Scholar,NaN,NaN,NaN
2,1907.0,"Killian, G.",G Killian,"G Killian - Annals of Otology, Rhinology & Lar...",Annals Of Otology,Xx. The Treatment Of Foreign Bodies In The Res...,The invitation of your President to deliver an...,https://journals.sagepub.com/doi/abs/10.1177/0...,Google Scholar,NaN,NaN,NaN
3,1917.0,"Benedetto, M.",M DI BENEDETTO,M DI BENEDETTO - REHABILITATION OF THE INJURED...,Rehabilitation Of The Injured …,Cranial Neuropathies: Electro-Diagnosis And Ma...,"CONCLUSION * Colonel Medical Corps, US Army (R...",https://upload.wikimedia.org/wikipedia/commons...,Google Scholar,NaN,NaN,NaN
4,1917.0,"Ritter, T.",TJ Ritter,"TJ Ritter - Nursing, 1917 - uk.i-scream.org",Nursing,Mother'S Remedies,╨╧*αí▒*ß>*■ *¡zV*|V*╠U*═U╬U╧U╨U╤U╥U╙U╘U╒U╓U╫U╪...,ftp://ftp.uk.i-scream.org/sites/gutenberg.org/...,Google Scholar,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
130,NaN,"Illenyi, J.","J Hirschberg, T Szende, P Koltai, A Illenyi","J Hirschberg, T Szende, P Koltai, A Illenyi - ...",NaN,"Pediatric Airway: Cry, Stridor And Cough",This multidisciplinary book describes 42 diffe...,https://books.google.com/books?hl=en&lr=&id=z1...,Google Scholar,NaN,NaN,NaN
131,NaN,"Coonar, F.","FC Wells, AS Coonar","FC Wells, AS Coonar - 2018 - books.google.com",NaN,Thoracic Surgical Techniques,This heavily revised second edition of this bo...,https://books.google.com/books?hl=en&lr=&id=nz...,Google Scholar,NaN,NaN,NaN
132,NaN,"Davis, E.",EP Davis,EP Davis - 1917 - books.google.com,NaN,Obstetric And Gynecologic Nursing,CHAPTER I DEFINITION OF PREGNANCY; THE GROWTH ...,https://books.google.com/books?hl=en&lr=&id=5g...,Google Scholar,NaN,NaN,NaN
133,NaN,"Sekouba, B.",B Sekouba,B Sekouba - 1996 - 34.250.91.188,NaN,Studies On The Cellular And Humoral Immune Res...,Amblyomma variegatum and Rhipicephalus appendi...,http://34.250.91.188:8080/xmlui/handle/1234567...,Google Scholar,NaN,NaN,NaN


### Duplicate Removal

In [44]:
start_count = len(google_df)
google_df = google_df.drop_duplicates(subset=google_df.columns, keep='first')
end_count = len(google_df)

duplicates_removed_count = end_count - start_count

print(f"{duplicates_removed_count} duplicates were removed.")


0 duplicates were removed.


In [48]:
# export for title and abstract review
google_df.to_csv(os.path.join(OUTPUT_PATH, "google_school_title_abstract_screen_start.csv"))

At this stage, after discussing with my supervisor and a librarian, it is decided that the 135 result search strategy would suffice. 
The results were exported to CSV and the URLs were accessed to access title and abstracts. 
Abstracts - where available - were transferred to the spreadsheet using Microsoft excel and exclusion criteria were applied.

## Bibliography

1.	Haddaway NR, Collins AM, Coughlin D, Kirk S. The Role of Google Scholar in Evidence Reviews and Its Applicability to Grey Literature Searching. Wray KB, editor. PLoS ONE. 2015 Sep 17;10(9):e0138237. 